In [1]:
!pip install -r requirements.txt

In [2]:
import os
import logging
import time
from datetime import datetime
from botocore.exceptions import ClientError

from strands import Agent, tool
from strands.hooks import (
    AgentInitializedEvent, 
    MessageAddedEvent, 
    AfterInvocationEvent,
    HookProvider, 
    HookRegistry
)
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

In [3]:
# Setup logging to see what the agent is doing
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("food-agent")

# Configuration
REGION = os.getenv('AWS_REGION', 'us-east-1')
USER_ID = "food-lover-001"  # User ID for short and long term memory using AWS Agent Core
SESSION_ID = f"food_chat_{datetime.now().strftime('%Y%m%d%H%M%S')}"

print(f"Region: {REGION}")
print(f"User ID: {USER_ID}")
print(f"Session ID: {SESSION_ID}")

Region: us-east-1
User ID: food-lover-001
Session ID: food_chat_20260224153028


In [4]:
# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "FoodAgentMemory"
memory_id = None

# Define memory strategy for food preferences
# Everything goes to short term memory first, then Agent Core moves preferences to long term memory (one time task)
strategies = [
    {
        StrategyType.USER_PREFERENCE.value: {
            "name": "FoodPreferences",
            "description": "Captures food preferences including cuisines, dietary restrictions, favorite dishes, and specific foods the user likes or dislikes",
            "namespaces": ["user/{actorId}/food_preferences"]
        }
    }
]

try:
    # Create memory resource
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=strategies,
        description="Memory for food recommendation agent - stores user food preferences",
        event_expiry_days=7,  # Keep preferences for a week
        max_wait=300,
        poll_interval=10
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # Memory already exists - retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"✅ Memory already exists. Using: {memory_id}")
    else:
        raise e

print(f"\n📝 Save this memory_id for future sessions: {memory_id}")

2026-02-24 15:30:31,327 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2026-02-24 15:30:31,413 - INFO - Initialized MemoryClient for control plane: us-east-1, data plane: us-east-1
2026-02-24 15:30:32,228 - ERROR - Failed to create memory: An error occurred (ValidationException) when calling the CreateMemory operation: Validation failed during CreateMemory: Memory with name FoodAgentMemory already exists
2026-02-24 15:30:32,564 - INFO - ✅ Memory already exists. Using: FoodAgentMemory-2SXptmCV1E



📝 Save this memory_id for future sessions: FoodAgentMemory-2SXptmCV1E


In [5]:
class FoodMemoryHookProvider(HookProvider):
    """Automatic memory management for food agent"""
    
    def __init__(self, memory_client: MemoryClient, memory_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load food preferences when agent starts"""
        try:
            actor_id = event.agent.state.get("actor_id")
            if not actor_id:
                logger.warning("Missing actor_id in agent state")
                return
            
            namespace = f"user/{actor_id}/food_preferences"
            
            # Retrieve stored food preferences (querying food preferences genres directory)
            preferences = self.memory_client.retrieve_memories(
                memory_id=self.memory_id,
                namespace=namespace,
                query="food preferences cuisines dietary restrictions favorites",
                top_k=3
            )
            
            if preferences:
                # Format preferences for context
                pref_texts = []
                for pref in preferences:
                    if isinstance(pref, dict):
                        content = pref.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                pref_texts.append(f"- {text}")
                
                if pref_texts:
                    context = "\n".join(pref_texts)
                    event.agent.system_prompt += f"\n\n## User's Food Preferences (from previous conversations):\n{context}"
                    logger.info(f"✅ Loaded {len(pref_texts)} food preferences")
            else:
                logger.info("No previous food preferences found - starting fresh!")
                    
        except Exception as e:
            logger.error(f"Error loading preferences: {e}")
    
    def on_after_invocation(self, event: AfterInvocationEvent):
        """Save conversation after each interaction"""
        try:
            messages = event.agent.messages
            # Save to memory after two interactions
            if len(messages) < 2:
                return
                
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                logger.warning("Missing actor_id or session_id")
                return
            
            # Get the last user message and assistant response
            user_msg = None
            assistant_msg = None
            
            for msg in reversed(messages):
                if msg["role"] == "assistant" and not assistant_msg:
                    content = msg.get("content", [])
                    if content and isinstance(content[0], dict) and "text" in content[0]:
                        assistant_msg = content[0]["text"]
                elif msg["role"] == "user" and not user_msg:
                    content = msg.get("content", [])
                    if content and isinstance(content[0], dict) and "text" in content[0]:
                        if "toolResult" not in content[0]:
                            user_msg = content[0]["text"]
                            break
            
            if user_msg and assistant_msg:
                # Save the conversation turn to short term memory
                self.memory_client.create_event(
                    memory_id=self.memory_id,
                    actor_id=actor_id,
                    session_id=session_id,
                    messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                )
                logger.info("💾 Saved conversation to memory")
                
        except Exception as e:
            logger.error(f"Error saving conversation: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)
        registry.add_callback(AfterInvocationEvent, self.on_after_invocation)
        logger.info("✅ Food memory hooks registered")

In [6]:
from ddgs import DDGS
from ddgs.exceptions import DDGSException, RatelimitException

@tool
def search_food(query: str, max_results: int = 5) -> str:
    """Search for food information, recipes, cuisines, or restaurant recommendations.
    
    Args:
        query: Search query about food (e.g., \"best authentic italian pasta recipes\")
        max_results: Maximum number of results to return
    
    Returns:
        Search results with food information
    """
    try:
        results = DDGS().text(f"{query} food recipe restaurant", region="us-en", max_results=max_results)
        if not results:
            return "No results found."
        
        formatted = []
        for i, r in enumerate(results, 1):
            formatted.append(f"{i}. {r.get('title', 'No title')}\n   {r.get('body', '')}")
        
        return "\n\n".join(formatted)
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Food search tool ready")

2026-02-24 15:31:02,740 - INFO - ✅ Food search tool ready


In [7]:
def create_food_agent(user_id: str, session_id: str):
    """Create a food recommendation agent with memory"""
    
    system_prompt = f"""You are a concise food assistant. Help users discover new foods & remember their preferences. Date: {datetime.today().strftime("%Y-%m-%d")}"""
    
    # Create memory hooks
    memory_hooks = FoodMemoryHookProvider(client, memory_id)
    
    # Create agent
    agent = Agent(
        name="FoodieBuddy",
        model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
        system_prompt=system_prompt,
        hooks=[memory_hooks],
        tools=[search_food],
        state={"actor_id": user_id, "session_id": session_id}
    )
    
    return agent

# Create the agent
food_agent = create_food_agent(USER_ID, SESSION_ID)
logger.info("✅ Food agent created with memory!")

import time
import logging
logger = logging.getLogger("food-agent")

def safe_invoke(agent, user_input, max_retries=3):
    """Invokes the agent with rate limiting and fallback mechanisms."""
    fallbacks = [
        "us.anthropic.claude-3-5-haiku-20241022-v1:0",
        "us.anthropic.claude-3-haiku-20240307-v1:0",
        "us.anthropic.claude-3-5-sonnet-20241022-v2:0"
    ]
    
    time.sleep(1) # Client-side rate limit spacing
    
    for attempt in range(max_retries):
        try:
            return agent(user_input)
        except Exception as e:
            error_str = str(e).lower()
            if "throttl" in error_str or "429" in error_str or "rate" in error_str:
                logger.warning(f"⚠️ Throttled (Attempt {attempt+1}/{max_retries}). Error: {str(e)[:50]}")
                if attempt < len(fallbacks):
                    fallback_model = fallbacks[attempt]
                    logger.info(f"🔄 Switching to fallback model: {fallback_model}")
                    agent.model = fallback_model
                    time.sleep(2 ** attempt)
                else:
                    raise e
            else:
                raise e


2026-02-24 15:31:06,722 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2026-02-24 15:31:06,767 - INFO - ✅ Food memory hooks registered
2026-02-24 15:31:07,740 - INFO - Retrieved 2 memories from namespace: user/food-lover-001/food_preferences
2026-02-24 15:31:07,741 - INFO - ✅ Loaded 2 food preferences
2026-02-24 15:31:07,741 - INFO - ✅ Food agent created with memory!


In [8]:
# Test the agent - logging is already enabled in the configuration cell
print("You: I really love spicy food, especially Thai cuisines! But I have a peanut allergy.")
print("\nAgent: ", end="")
safe_invoke(food_agent, "I really love spicy food, especially Thai cuisines! But I have a peanut allergy.")

You: I really love spicy food, especially Thai cuisines! But I have a peanut allergy.

Agent: 

2026-02-24 15:31:16,860 - INFO - Creating Strands MetricsClient


I understand that you love spicy Thai food but have a peanut allergy, which is an important consideration. Let me help you find some delicious and safe Thai dishes that you can enjoy.
Tool #1: search_food


2026-02-24 15:31:21,278 - WARNING - Impersonate 'edge_127' does not exist, using 'random'
2026-02-24 15:31:21,793 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=Thai%20spicy%20dishes%20without%20peanuts%20safe%20for%20peanut%20allergy%20food%20recipe%20restaurant 200
2026-02-24 15:31:22,573 - INFO - response: https://grokipedia.com/api/typeahead?query=Thai+spicy+dishes+without+peanuts+safe+for+peanut+allergy+food+recipe+restaurant&limit=1 200
2026-02-24 15:31:23,407 - INFO - response: https://search.brave.com/search?q=Thai+spicy+dishes+without+peanuts+safe+for+peanut+allergy+food+recipe+restaurant&source=web 200


Great news! Here are some recommendations for spicy Thai dishes that are safe for someone with a peanut allergy:

1. Modified Pad Thai: You can enjoy Pad Thai without peanuts. Some tips:
   - Ask for a peanut-free version
   - Use alternatives like sriracha for extra spiciness
   - Request tofu or chicken as a protein

2. Safety Tips for Thai Dining:
   - Always communicate your peanut allergy clearly to the restaurant staff
   - Be cautious about cross-contamination in the kitchen
   - Ask about their cooking oils and preparation methods

Some spicy Thai dishes to try that typically don't contain peanuts:
- Tom Yum Soup
- Som Tam (Papaya Salad - just ensure no peanuts)
- Larb (Spicy Meat Salad)
- Green or Red Thai Curry
- Pad Kee Mao (Drunken Noodles)

Pro tip: Always double-check ingredients and inform the restaurant about your peanut allergy to ensure your safety while enjoying delicious, spicy Thai cuisine!

Would you like me to help you find a specific peanut-free, spicy Thai reci

2026-02-24 15:31:31,322 - INFO - Created event: 0000001771927290974#76a53b14
2026-02-24 15:31:31,324 - INFO - 💾 Saved conversation to memory


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Great news! Here are some recommendations for spicy Thai dishes that are safe for someone with a peanut allergy:\n\n1. Modified Pad Thai: You can enjoy Pad Thai without peanuts. Some tips:\n   - Ask for a peanut-free version\n   - Use alternatives like sriracha for extra spiciness\n   - Request tofu or chicken as a protein\n\n2. Safety Tips for Thai Dining:\n   - Always communicate your peanut allergy clearly to the restaurant staff\n   - Be cautious about cross-contamination in the kitchen\n   - Ask about their cooking oils and preparation methods\n\nSome spicy Thai dishes to try that typically don't contain peanuts:\n- Tom Yum Soup\n- Som Tam (Papaya Salad - just ensure no peanuts)\n- Larb (Spicy Meat Salad)\n- Green or Red Thai Curry\n- Pad Kee Mao (Drunken Noodles)\n\nPro tip: Always double-check ingredients and inform the restaurant about your peanut allergy to ensure your safety while enjoying

In [9]:
print("\nYou: Can you give me a recommendation for tonight?")
print("\nAgent: ", end="")
safe_invoke(food_agent, "Can you give me a recommendation for tonight?")


You: Can you give me a recommendation for tonight?

Agent: Let me search for a delicious, spicy Thai dish that's safe for your peanut allergy.
Tool #2: search_food


2026-02-24 15:32:13,022 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=Best%20spicy%20Thai%20Tom%20Yum%20soup%20recipe%20without%20peanuts%20food%20recipe%20restaurant 200
2026-02-24 15:32:13,034 - INFO - response: https://grokipedia.com/api/typeahead?query=Best+spicy+Thai+Tom+Yum+soup+recipe+without+peanuts+food+recipe+restaurant&limit=1 200
2026-02-24 15:32:14,199 - INFO - response: https://yandex.com/search/site/?text=Best+spicy+Thai+Tom+Yum+soup+recipe+without+peanuts+food+recipe+restaurant&web=1&searchid=8702184 200


I recommend Tom Yum Soup for tonight! Here's why it's perfect for you:
- Naturally peanut-free
- Extremely spicy (meeting your love for heat)
- Authentic Thai cuisine
- Typically made with shrimp (but can be made with chicken or tofu)

Key ingredients that make Tom Yum amazing:
- Lemongrass
- Kaffir lime leaves
- Chili peppers
- Shrimp (or protein of choice)
- Mushrooms
- Lime juice
- Fish sauce

Cooking tips:
- Use Mae Ploy Tom Yum paste for authentic flavor
- Add extra chili if you want more heat
- Double-check all ingredients to ensure no peanut contamination

Would you like me to help you find a specific Tom Yum recipe that you can make at home safely?

2026-02-24 15:32:20,541 - INFO - Created event: 0000001771927340193#68a54fb5
2026-02-24 15:32:20,542 - INFO - 💾 Saved conversation to memory


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "I recommend Tom Yum Soup for tonight! Here's why it's perfect for you:\n- Naturally peanut-free\n- Extremely spicy (meeting your love for heat)\n- Authentic Thai cuisine\n- Typically made with shrimp (but can be made with chicken or tofu)\n\nKey ingredients that make Tom Yum amazing:\n- Lemongrass\n- Kaffir lime leaves\n- Chili peppers\n- Shrimp (or protein of choice)\n- Mushrooms\n- Lime juice\n- Fish sauce\n\nCooking tips:\n- Use Mae Ploy Tom Yum paste for authentic flavor\n- Add extra chili if you want more heat\n- Double-check all ingredients to ensure no peanut contamination\n\nWould you like me to help you find a specific Tom Yum recipe that you can make at home safely?"}]}, metrics=EventLoopMetrics(cycle_count=4, tool_metrics={'search_food': ToolMetrics(tool={'toolUseId': 'tooluse_NEUZdNo70Gw0EzlGYUEO8q', 'name': 'search_food', 'input': {'query': 'Best spicy Thai Tom Yum soup recipe without p

In [10]:
# Check SHORT-TERM MEMORY (Raw Conversations)
print("\nSHORT-TERM MEMORY (Raw Conversations)")
print("=" * 60)
events = client.list_events(memory_id=memory_id, actor_id=USER_ID, session_id=SESSION_ID)
if events:
    for i, event in enumerate(events, 1):
        print(f"\n--- Event {i} ---")
        print(event)
else:
    print("No events found. Memory ID:", memory_id)


SHORT-TERM MEMORY (Raw Conversations)


2026-02-24 15:32:39,325 - INFO - Retrieved total of 2 events



--- Event 1 ---
{'memoryId': 'FoodAgentMemory-2SXptmCV1E', 'actorId': 'food-lover-001', 'sessionId': 'food_chat_20260224153028', 'eventId': '0000001771927340193#68a54fb5', 'eventTimestamp': datetime.datetime(2026, 2, 24, 15, 32, 20, 193000, tzinfo=tzlocal()), 'payload': [{'conversational': {'content': {'text': 'Can you give me a recommendation for tonight?'}, 'role': 'USER'}}, {'conversational': {'content': {'text': "I recommend Tom Yum Soup for tonight! Here's why it's perfect for you:\n- Naturally peanut-free\n- Extremely spicy (meeting your love for heat)\n- Authentic Thai cuisine\n- Typically made with shrimp (but can be made with chicken or tofu)\n\nKey ingredients that make Tom Yum amazing:\n- Lemongrass\n- Kaffir lime leaves\n- Chili peppers\n- Shrimp (or protein of choice)\n- Mushrooms\n- Lime juice\n- Fish sauce\n\nCooking tips:\n- Use Mae Ploy Tom Yum paste for authentic flavor\n- Add extra chili if you want more heat\n- Double-check all ingredients to ensure no peanut conta

In [11]:
# Check LONG-TERM MEMORY (Background Processed Preferences)
print("\nLONG-TERM MEMORY PREFERENCES:")
print("=" * 50)
try:
    preferences = client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"user/{USER_ID}/food_preferences",
        query="food preferences cuisines dietary restrictions favorites",
        top_k=3
    )

    if preferences:
        for i, pref in enumerate(preferences, 1):
            if isinstance(pref, dict):
                content = pref.get('content', {})
                if isinstance(content, dict):
                    text = content.get('text', '')
                    if text:
                        print(f"{i}. {text}")
    else:
        print("No preferences extracted yet. It could take 30-60 seconds to process in the background.")
except Exception as e:
    print(f"Could not retrieve from long-term memory: {e}")


LONG-TERM MEMORY PREFERENCES:


2026-02-24 15:32:56,063 - INFO - Retrieved 2 memories from namespace: user/food-lover-001/food_preferences


1. {"context":"The user explicitly mentioned having a peanut allergy, which is a critical dietary restriction.","preference":"Has a peanut allergy","categories":["food","health","dietary restrictions","allergies"]}
2. {"context":"The user explicitly stated that they really love spicy food, especially Thai cuisines.","preference":"Loves spicy food, especially Thai cuisine","categories":["food","cuisine","Thai food","spicy food"]}


In [ ]:
# Test the agent - logging is already enabled in the configuration cell
print("You: I really love spicy food, especially Thai cuisines! But I have a peanut allergy.")
print("\nAgent: ", end="")
safe_invoke(food_agent, "I really love spicy food, especially Thai cuisines! But I have a peanut allergy.")

print("\nYou: Can you give me a recommendation for tonight?")
print("\nAgent: ", end="")
safe_invoke(food_agent, "Can you give me a recommendation for tonight?")

print(f"\nSession ID: {SESSION_ID}")

# Check SHORT-TERM MEMORY (Raw Conversations)
print("\nSHORT-TERM MEMORY (Raw Conversations)")
print("=" * 60)
events = client.list_events(memory_id=memory_id, actor_id=USER_ID, session_id=SESSION_ID)
if events:
    for i, event in enumerate(events, 1):
        print(f"\n--- Event {i} ---")
        print(event)
else:
    print("No events found. Memory ID:", memory_id)

# Check LONG-TERM MEMORY (Background Processed Preferences)
print("\nLONG-TERM MEMORY PREFERENCES:")
print("=" * 50)
try:
    preferences = client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"user/{USER_ID}/food_preferences",
        query="food preferences cuisines dietary restrictions favorites",
        top_k=3
    )

    if preferences:
        for i, pref in enumerate(preferences, 1):
            if isinstance(pref, dict):
                content = pref.get('content', {})
                if isinstance(content, dict):
                    text = content.get('text', '')
                    if text:
                        print(f"{i}. {text}")
    else:
        print("No preferences extracted yet. It could take 30-60 seconds to process in the background.")
except Exception as e:
    print(f"Could not retrieve from long-term memory: {e}")

## Simulating a New Session
Let's simulate coming back later. We will create a new agent instance (like opening the app again) with the same `USER_ID` and memory, but a new `SESSION_ID`. We'll see if it remembers the user's preferences.

In [12]:
# Wait a few seconds to let background processes digest the memory
import time
print("Waiting for memory processing to finish...")
time.sleep(10)

# Simulate a new session
SESSION_ID_2 = f"food_chat_{datetime.now().strftime('%Y%m%d%H%M%S')}_2"
print(f"New Session ID: {SESSION_ID_2}")

# Create a new agent instance for the new session
food_agent_2 = create_food_agent(USER_ID, SESSION_ID_2)


Waiting for memory processing to finish...


2026-02-24 15:33:13,954 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2026-02-24 15:33:14,031 - INFO - ✅ Food memory hooks registered


New Session ID: food_chat_20260224153313_2


2026-02-24 15:33:14,464 - INFO - Retrieved 2 memories from namespace: user/food-lover-001/food_preferences
2026-02-24 15:33:14,465 - INFO - ✅ Loaded 2 food preferences


In [13]:
# Test if the new agent remembers the preferences (peanut allergy, spicy food, Thai)
print("You: I'm hungry! What should I get for dinner tonight?")
print("\nAgent: ", end="")
safe_invoke(food_agent_2, "I'm hungry! What should I get for dinner tonight? Do you remember what I like and can't eat?")


You: I'm hungry! What should I get for dinner tonight?

Agent: Based on your previous preferences and dietary restrictions, I recall two important details:
1. You have a peanut allergy, so I'll be careful to avoid recommending any dishes with peanuts
2. You love spicy food, especially Thai cuisine

Let me help you find a delicious and safe dinner option! I'll search for some spicy Thai dishes that should be peanut-free.
Tool #1: search_food


2026-02-24 15:33:42,602 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=spicy%20Thai%20main%20dishes%20without%20peanuts%20food%20recipe%20restaurant 200
2026-02-24 15:33:43,005 - INFO - response: https://grokipedia.com/api/typeahead?query=spicy+Thai+main+dishes+without+peanuts+food+recipe+restaurant&limit=1 200
2026-02-24 15:33:44,428 - INFO - response: https://yandex.com/search/site/?text=spicy+Thai+main+dishes+without+peanuts+food+recipe+restaurant&web=1&searchid=6716745 200


Given the search results and your preferences, I recommend trying a spicy Thai dish like Pad Thai or another spicy Thai main course. Just be sure to specifically request "no peanuts" if ordering at a restaurant due to your allergy.

A few suggestions:
- Tom Yum soup with chicken or shrimp (very spicy)
- Spicy Thai basil chicken (Pad Kra Pao Gai)
- Green curry with vegetables and protein

Would you like me to help you find a specific recipe or restaurant that can accommodate your spicy food preference and peanut allergy?

2026-02-24 15:33:49,662 - INFO - Created event: 0000001771927429295#7213f5e6
2026-02-24 15:33:49,663 - INFO - 💾 Saved conversation to memory


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Given the search results and your preferences, I recommend trying a spicy Thai dish like Pad Thai or another spicy Thai main course. Just be sure to specifically request "no peanuts" if ordering at a restaurant due to your allergy.\n\nA few suggestions:\n- Tom Yum soup with chicken or shrimp (very spicy)\n- Spicy Thai basil chicken (Pad Kra Pao Gai)\n- Green curry with vegetables and protein\n\nWould you like me to help you find a specific recipe or restaurant that can accommodate your spicy food preference and peanut allergy?'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'search_food': ToolMetrics(tool={'toolUseId': 'tooluse_nJ9P8bMfD2rOihtr7kWXiL', 'name': 'search_food', 'input': {'query': 'spicy Thai main dishes without peanuts', 'max_results': 5}}, call_count=1, success_count=1, error_count=0, total_time=2.37985897064209)}, cycle_durations=[4.787191867828369], agent_invocations=[Age

## AgentCore Identity & Agent Core Policy

While the memory system enables the agent to remember user preferences over time, **AgentCore Identity** and **Agent Core Policy** work together to ensure that the agent operates securely and only has access to authorized tools and functions.

### How Identity and Policy Work Together

1. **AgentCore Identity (Authentication):**
   - AgentCore Identity provides a secure way for agents to authenticate and obtain credentials for accessing external systems, APIs, or AWS resources.
   - Using the `@requires_access_token` or `@requires_iam_access_token` decorators, agents can dynamically fetch short-lived OAuth 2.0 or AWS IAM tokens right before executing a tool.
   - This prevents secrets from being hardcoded in the code and delegates the complex identity federation logic to AWS STS and Identity Providers.

2. **Agent Core Policy (Authorization & Governance):**
   - Once an agent is authenticated and attempts to call a tool or API, **Agent Core Policy** intercepts the request at the *Bedrock Gateway* level (infrastructure layer).
   - Policies (written in Cedar or Natural Language) evaluate the agent's identity, the specific tool being requested, and the context of the action.
   - For example, an Agent Core Policy might state: *"Only the 'FoodieBuddy' agent identity is allowed to access the `book_restaurant` tool, and only if the reservation cost is under $100."*
   - If the policy denies the action, the Gateway blocks the request deterministically, completely independently of the agent's LLM code.

By combining Identity (who the agent is and how it authenticates) and Policy (what the agent's identity is strictly allowed to do), developers can build secure, enterprise-ready agents.

Below is an example simulating how a mock tool might request a Machine-to-Machine (M2M) IAM access token via Identity before making a request. (Note: A real-world scenario would require setting up outbound OAuth/JWT federation in your AWS account).

In [1]:
# -------------------------------------------------------------------------------------
# DEMO: AgentCore Identity Integration using `@requires_iam_access_token`
# -------------------------------------------------------------------------------------
# Note: To run this against a real API, AWS IAM Outbound Web Identity Federation
# must be enabled in your account. The decorator automatically fetches a signed JWT
# from STS before the tool function executes.

from bedrock_agentcore.identity.auth import requires_iam_access_token
from strands import tool

@tool
@requires_iam_access_token(
    audience=["https://api.example-restaurant.com/v1/*"], 
    signing_algorithm="ES384",
    duration_seconds=300
)
async def book_restaurant(restaurant_id: str, date: str, time: str, guests: int, *, access_token: str) -> str:
    """Book a table at a restaurant. Requires an IAM Access Token injected by AgentCore Identity.
    
    Args:
        restaurant_id: The ID of the restaurant.
        date: The date for the booking (YYYY-MM-DD).
        time: The time for the booking (HH:MM).
        guests: The number of guests.
        access_token: The signed JWT access token injected by the decorator. (DO NOT PASS THIS IN PROMPT)
    """
    
    # In a real environment, you would use this access_token in the Authorization header 
    # to authenticate against the downstream API.
    
    # Example (mocked):
    # headers = {"Authorization": f"Bearer {access_token}"}
    # response = requests.post("https://api.example-restaurant.com/v1/book", headers=headers, json={"restaurant_id": restaurant_id, "date": date, ...})
    
    print(f"[Identity Log] Successfully fetched JWT Access Token (length: {len(access_token)}) from AWS STS.")
    print(f"[Identity Log] Authorizing request to book {restaurant_id} using injected token...")
    
    # IMPORTANT: 
    # At this point in a real Bedrock AgentCore environment, the request would pass 
    # through the Agent Core Gateway where the Agent Core Policy evaluating the agent's 
    # identity and permissions deterministically allows or blocks this action based on Cedar rules.
    
    return f"Successfully booked {guests} guests at {restaurant_id} for {date} at {time} using IAM JWT authentication."

# In a real application, you would pass `book_restaurant` into your Agent's `tools` list!
print("Identity mock tool registered!")


Identity mock tool registered!
